# 1.  Importing Packages

In [1]:
# Import PyTorch
import torch
from torch import nn

# Import torchvision
import torchvision
from torchvision import datasets
from torchvision import transforms 
from torchvision.transforms import ToTensor # covert image or np array to tensors

# Import matplotlib for visualization
import matplotlib.pyplot as plt

# Check versions
print(torch.__version__)
print(torchvision.__version__)

1.8.1+cu102
0.9.1+cu102


In [2]:
!pip3 install torchmetrics

In [3]:
from torchmetrics.classification import ConfusionMatrix

In [4]:
!pip3 install mlxtend

In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [6]:
# Setup device-agnostic code
import torch 
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

# 2. Load the torchvision.datasets.MNIST() train and test datasets

In [ ]:
train_dataset = datasets.MNIST(
                root = '.',
                train = True,
                download = True,
                transform = transforms.ToTensor()) #convert the image or numpy data to tensors 

test_dataset = datasets.MNIST(
                root = '.',
                train = False,
                download = True,
                transform = transforms.ToTensor()) #convert the image or numpy data to tensors) #donot convert class labels to tensor)

In [ ]:
#finding length of train and test dataset
len(train_dataset), len(test_dataset)

In [ ]:
#checking the shape of the data
type(train_dataset[0])

In [ ]:
#print the sample 
train_dataset[0] #so we can see data is in the form of image as array and label

In [ ]:
#print image and label
print(f"Image: \n {train_dataset[0][0]}")
print(f"Label: \n {train_dataset[0][1]}")

In [ ]:
print(f"Image: \n {train_dataset[1][0]}")
print(f"Label: \n {train_dataset[1][1]}")

In [ ]:
image = train_dataset[1][0]
label = train_dataset[1][1]
print(f"Image shape:{image.shape} --> [Color channel first format], \nlabel:{label} -->[no shape because its an integer]")

# 3.   Visualize at least 5 different samples of the MNIST training dataset.

In [ ]:
#getting the class names of train dataset
class_names = train_dataset.classes
class_names

In [ ]:
#getting the index of the train dataset
class_to_idx = train_dataset.class_to_idx
class_to_idx

In [ ]:
import matplotlib.pyplot as plt

for i in range(5):
    img = train_dataset[i][0]
    print(f"Image Shape : {image.shape}")

    img_squeeze=img.squeeze() #squeezing the image as the original image is in format color channel first. but matplotlib expects colorchannels last
    print(f"img_squeeze Shape : {img_squeeze.shape}")

    label = train_dataset[i][1]
    plt.figure(figsize=(3,3))
    plt.imshow(img_squeeze) 

    plt.title(label)
    plt.axis(False);

In [ ]:
#plotting more images using randomness

torch.manual_seed(42)
fig = plt.figure(figsize =(9,9))
rows,cols = 4,4
for i in range(1, rows*cols+1):
    random_idx = torch.randint(0, len(train_dataset), size =[1]).item()
    img,label = train_dataset[random_idx]
    fig.add_subplot(rows,cols,i);
    plt.imshow(img.squeeze(), cmap ='gray')
    plt.title(class_names[label]);
    plt.axis(False); #removing grid


# 4.  Turn the MNIST train and test datasets into dataloaders using torch.utils.data.DataLoader, set the batch_size=32.

In [ ]:
#turn the dataset into dataloaders

from torch.utils.data import DataLoader
train_dataloader = DataLoader(dataset = train_dataset,
                             batch_size = 32,
                             shuffle = True)

In [ ]:
#turn the dataset into dataloaders

from torch.utils.data import DataLoader
test_dataloader = DataLoader(dataset = test_dataset,
                             batch_size = 32,
                             shuffle = False)

In [ ]:
print(f"Length of train_dataloader :{len(train_dataloader)},Length of test_dataloader :{len(test_dataloader)}")

# 5. Building CNN- TinyVGG model fitting on the MNIST dataset.

In [ ]:
#check the input and output shape before building the model

train_features_batch, train_labels_batch = next(iter(train_dataloader))
#next(iter(dataloader)) you can only access a single batch of data, this is more effecient than for loop if you want to view a single batch of data
train_features_batch.shape, train_labels_batch.shape



In [ ]:
test_features_batch, test_labels_batch = next(iter(test_dataloader))
#next(iter(dataloader)) you can only access a single batch of data, this is more effecient than for loop if you want to view a single batch of data
test_features_batch.shape, test_labels_batch.shape

###### The input image shape is 1 (inputshape denotes the number of color channels ). output shape is length of the class names

In [ ]:
#creating a convolutional neural network
class MNISTModelV2(nn.Module):
    """
    Model architecture copying TinyVGG from: 
    https://poloclub.github.io/cnn-explainer/
    """
    def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
        super().__init__()
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape, 
                      out_channels=hidden_units, 
                      kernel_size=3, # how big is the square that's going over the image?
                      stride=1, # default
                      padding=1),# options = "valid" (no padding) or "same" (output has same shape as input) or int for specific number 
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units, 
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,
                         stride=2) # default stride value is same as kernel_size
        )
        self.block_2 = nn.Sequential(
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(hidden_units, hidden_units, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            # Where did this in_features shape come from? 
            # It's because each layer of our network compresses and changes the shape of our inputs data.
            nn.Linear(in_features=hidden_units*7*7, 
                      out_features=output_shape)
        )
    
    def forward(self, x: torch.Tensor):
        x = self.block_1(x)
        # print(x.shape)
        x = self.block_2(x)
        # print(x.shape)
        x = self.classifier(x)
        # print(x.shape)
        return x

In [ ]:
torch.manual_seed(42)
tinyvgg = MNISTModelV2(input_shape=1, hidden_units=10, output_shape=len(class_names)).to(device)
tinyvgg

In [ ]:
#checking the current state of the model
tinyvgg.state_dict()

# 6. Train the model for 5 epochs on CPU 

In [ ]:
#set up accuracy, loss function and optimizer

def accuracy_fn(y_true, y_pred):
    """Calculates accuracy between truth labels and predictions.

    Args:
        y_true (torch.Tensor): Truth labels for predictions.
        y_pred (torch.Tensor): Predictions to be compared to predictions.

    Returns:
        [torch.float]: Accuracy value between y_true and y_pred, e.g. 78.45
    """
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct / len(y_pred)) * 100
    return acc

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(params=tinyvgg.parameters(), lr =0.1)


In [ ]:
#Creating a function to time our experiments

from timeit import default_timer as timer 
def print_train_time(start: float,
                     end: float, 
                     device: torch.device = None):
    #Prints difference between start and end time
    total_time = end - start
    print(f"Train time on {device}: {total_time:.3f} seconds")
    return total_time


In [ ]:
#import tqdm for progress bar
from tqdm.auto import tqdm

#set the seed and start the timer
torch.manual_seed(42)
train_time_start_on_cpu = timer()

#set the number of epochs
epochs = 5

#creating training and test loop

for epoch in tqdm(range(epochs)):
    print(f"Epoch: {epoch}")
    
    #set up train loss 
    train_loss = 0 #to calculate train loss step2
    
    #loop to loop through training batches
    
    for batch,(X,y) in enumerate(train_dataloader):
        
        1.#training model
        tinyvgg.train()
        
        2. #forward pass
        y_pred = tinyvgg(X)  
        
        3.#calculate the loss {per batch}
        loss = loss_fn(y_pred,y)
        train_loss += loss #accumulate the train loss
        
        4. #Optimizer zero grad
        optimizer.zero_grad()
        
        #5. Loss backward
        loss.backward()
        
        #6. Optimizer step
        optimizer.step()
        
        #print out whats happening
        if batch % 400 == 0:
            print(f"Looked at {batch * len(X)}/ {len(train_dataloader.dataset)}")
        
    
    #Divide total train loss by length of train dataloader
    
    train_loss /= len(train_dataloader)
    
    #testing
    test_loss, test_acc = 0,0
    tinyvgg.eval()
    
    with torch.no_grad():
        for X_test, y_test in test_dataloader:
        
            #1. forward pass
            test_pred = tinyvgg(X_test)

            #2. calculate loss (accumulately)
            test_loss += loss_fn(test_pred, y_test)

            #3. calculate accuracy
            test_acc += accuracy_fn(y_true = y_test, y_pred = test_pred.argmax(dim=1))
        
        #calculate test loss average per batch
        test_loss /= len(test_dataloader)

        #calculate test accuracy average per batch
        test_acc /= len(test_dataloader)
        
    #print out whats happening
    print(f"\nTrain loss:{train_loss:.4f} | Test loss:{test_loss:.4f} | Test Acc: {test_acc:.2f}%")


#calculate training time
train_time_end_on_cpu = timer()
total_train_time_model_0 =  print_train_time(start=train_time_start_on_cpu,
                                            end = train_time_end_on_cpu,
                                            device = str(next(tinyvgg.parameters()).device))
    

In [ ]:
#Evaluating the model on test dataset

torch.manual_seed(42)

def eval_model(model: torch.nn.Module,
              data_loader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              accuracy_fn,
            device=device):
    
    loss, acc = 0,0
    model.eval()
    
    with torch.no_grad():
        for X,y in tqdm(data_loader):
            
            #Make predictions
            y_pred = model(X)
            
            #Accumulate the loss and acc values per batch
            loss += loss_fn(y_pred,y)
            acc += accuracy_fn(y_true=y,
                              y_pred=y_pred.argmax(dim=1))
            # Scale loss and acc to find the average loss/acc per batch
        loss /= len(data_loader)
        acc /= len(data_loader)

    return {"model_name": model.__class__.__name__, # only works when model was created with a class
          "model_loss": loss.item(),
          "model_acc": acc}

In [ ]:
# Calculate tinyvgg results on test dataset
tinyvgg_results = eval_model(model=tinyvgg,
                             data_loader=test_dataloader,
                             loss_fn=loss_fn, 
                             accuracy_fn=accuracy_fn)
tinyvgg_results

# 7.  Make predictions using your trained model and visualize at least 5 of them comparing the prediciton to the target label

###### Method 1

In [ ]:
num_to_plot = 5
for i in range(num_to_plot):
    
    #get images and labels from test data
    img = test_dataset[i][0]
    label= test_dataset[i][1]
    
    #make prediction on image
    model_pred_logits = tinyvgg(img.unsqueeze(dim=0).to(device))
    model_pred_probabilities= torch.softmax(model_pred_logits, dim=1)
    model_pred_labels= torch.argmax(model_pred_probabilities, dim=1)
    
    #plot the image and prediction
    plt.figure()
    plt.imshow(img.squeeze(),cmap='gray')
    plt.title(f"Truth:{label}| Pred: {model_pred_labels.cpu().item()}")
    plt.axis(False);
    

#### Method 2

In [ ]:
#making prediction
def make_predictions(model: torch.nn.Module, data: list, device: torch.device = device):
    pred_probs = []
    model.eval()
    with torch.no_grad():
        for sample in data:
            # Prepare sample
            sample = torch.unsqueeze(sample, dim=0).to(device) # Add an extra dimension and send sample to device

            # Forward pass (model outputs raw logit)
            pred_logit = model(sample)

            # Get prediction probability (logit -> prediction probability)
            pred_prob = torch.softmax(pred_logit.squeeze(), dim=0)

            # Get pred_prob off GPU for further calculations
            pred_probs.append(pred_prob.cpu())
            
    # Stack the pred_probs to turn list into a tensor
    return torch.stack(pred_probs)

In [ ]:
#taking random samples from test_dataset
import random
random.seed(42)
test_samples = []
test_labels = []
for sample, label in random.sample(list(test_dataset), k=9):
    test_samples.append(sample)
    test_labels.append(label)

# View the first test sample shape and label
print(f"Test sample image shape: {test_samples[0].shape}\nTest sample label: {test_labels[0]} ({class_names[test_labels[0]]})")

In [ ]:
# Make predictions on test samples
pred_probs= make_predictions(model=tinyvgg, 
                             data=test_samples)

# View first two prediction probabilities list
pred_probs[:2]

In [ ]:
# Turn the prediction probabilities into prediction labels by taking the argmax()
pred_classes = pred_probs.argmax(dim=1)
pred_classes

In [ ]:
# Plot predictions
plt.figure(figsize=(9, 9))
nrows = 3
ncols = 3
for i, sample in enumerate(test_samples):
    # Create a subplot
    plt.subplot(nrows, ncols, i+1)

    # Plot the target image
    plt.imshow(sample.squeeze(), cmap="gray")

    # Find the prediction label (in text form, e.g. "Sandal")
    pred_label = class_names[pred_classes[i]]

    # Get the truth label (in text form, e.g. "T-shirt")
    truth_label = class_names[test_labels[i]] 

    # Create the title text of the plot
    title_text = f"Pred: {pred_label} | Truth: {truth_label}"
  
  # Check for equality and change title colour accordingly
    if pred_label == truth_label:
        plt.title(title_text, fontsize=10, c="g") # green text if correct
    else:
        plt.title(title_text, fontsize=10, c="r") # red text if wrong
    plt.axis(False);

# 8. Plot a confusion matrix comparing your model's predictions to the truth labels

In [ ]:
#get the class names
class_names = train_dataset.classes
class_names

In [ ]:
#Earlier we only predicted for few test samples,
#inorder to plot confusion matrix we need to run prediction loop for entire test dataset
# Import tqdm.auto
from tqdm.auto import tqdm 


# 1. Make predictions with trained model
y_preds = []
tinyvgg.eval()
with torch.inference_mode():
    for batch,(X, y) in tqdm(enumerate(test_dataloader)):
        # Send the data and targets to target device
        X, y = X.to(device), y.to(device)
        # Do the forward pass
        y_logit = tinyvgg(X)
        # Turn predictions from logits -> prediction probabilities -> prediction labels
        y_pred = torch.softmax(y_logit.squeeze(), dim=0).argmax(dim=1)
        # Put prediction on CPU for evaluation
        y_preds.append(y_pred.cpu())

# Concatenate list of predictions into a tensor
# print(y_preds)
y_pred_tensor = torch.cat(y_preds)
y_pred_tensor
     

In [ ]:
test_dataset.targets[:10],y_pred_tensor[:10]

In [ ]:
from torchmetrics import ConfusionMatrix
from mlxtend.plotting import plot_confusion_matrix

# 2. Setup confusion instance and compare predictions to targets
confmat = ConfusionMatrix(num_classes=len(class_names), task ='multiclass')
confmat_tensor = confmat(preds=y_pred_tensor,
                         target=test_dataset.targets)

# 3. Plot the confusion matrix
fig, ax = plot_confusion_matrix(
    conf_mat=confmat_tensor.numpy(), # matplotlib likes working with numpy
    class_names=class_names,
    figsize=(10, 7)
)

# 9. Save the model and make prediction on the trained model

In [ ]:
from pathlib import Path

#create model directory path

MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents = True,
                exist_ok = True)

#Create model save
MODEL_NAME = "03_pytorch_computer_vision_EXERCISE.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

#save the model state dict
print(f"Saving model to : {MODEL_SAVE_PATH}")
torch.save(obj=tinyvgg.state_dict(),
          f =MODEL_SAVE_PATH )

In [ ]:
#Create a new instance
torch.manual_seed(42)

loaded_tinyvgg = MNISTModelV2(input_shape=1,
                             hidden_units = 10,
                             output_shape=len(class_names))

#load in the save state_dict()
loaded_tinyvgg.state_dict(torch.load(f=MODEL_SAVE_PATH))

#send the model to the target device

loaded_tinyvgg.to(device)

In [ ]:
tinyvgg_results

In [ ]:
#evaluate the loaded model

torch.manual_seed(42)

loaded_tinyvgg_results = eval_model(
                            model=loaded_tinyvgg,
                             data_loader=test_dataloader,
                             loss_fn=loss_fn, 
                             accuracy_fn=accuracy_fn)

In [ ]:
loaded_tinyvgg_results